In [ ]:
import os
import sys
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../acquire")
import ipywidgets as widgets

from HUGS.Processing import search

from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

In [ ]:
# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Start with uploading and processing of the data

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
hugs_url = "https://hugs.acquire-aaai.com"
user = User(username="gareth", identity_url="%s/identity" % base_url)
response = user.request_login()

Login by visiting: https://login.acquire-aaai.com?id=a0-a6/61.4d.95.5d
(please check that this page displays the message 'black cats stand happily')


Check we're logged in

In [ ]:
user.wait_for_login()

True

In [ ]:
filename = "bsd.picarro.1minute.248m.dat"
dir_path = os.path.abspath("")
test_data = "../../test/data/proc_test_data/CRDS"
filepath = os.path.join(dir_path, test_data, filename)

hugs = Service(service_url="%s/hugs" % base_url)

creds = StorageCreds(user=user, service_url="%s/storage" % base_url)
drive = Drive(creds=creds, name="test_drive")
filemeta = drive.upload(filepath)
par = PAR(location=filemeta.location(), user=user)

par_secret = hugs.encrypt_data(par.secret())
auth = Authorisation(resource="process", user=user)

In [ ]:
def upload_file(authenticated_user, filepath):
    """ Upload a file to the object store
    
        Args:
            filepath (str): Path to file to upload
        Returns:
            None    
    """
    creds = StorageCreds(user=authenticated_user, service_url="%s/storage" % base_url)
    drive = Drive(creds=creds, name="test_drive")
    filemeta = drive.upload(filepath)
    par = PAR(location=filemeta.location(), user=user)

    par_secret = hugs.encrypt_data(par.secret())
    auth = Authorisation(resource="process", user=user)

In [ ]:
def get_CRDS_path(filename):
    dir_path = os.path.abspath("")
    test_data = "../../test/data/proc_test_data/CRDS"
    return os.path.join(dir_path, test_data, filename)

### Upload files for processing


Upload Bilsdale data

In [ ]:
bsd_path = get_CRDS_path("bsd.picarro.1minute.248m.dat")
upload_file(authenticated_user=user, filepath=bsd_path)


NameError: name 'get_CRDS_path' is not defined

Clear the Datasources from CRDS and GC objects (currently Datasource UUIDs are just being randomly generated and we get multiple Datasources holding the same data)

In [ ]:
response = hugs.call_function(function="clear_datasources", args={})

In [ ]:
args = {"authorisation": auth.to_data(),
            "par": {"data": par.to_data()},
            "par_secret": {"data": par_secret},
            "data_type": "CRDS"}


response = hugs.call_function(function="process", args=args)

In [ ]:
print(response)

{'results': ['7272bd9a-62e7-4c12-a9b2-30c2e8e136f0', '24373428-1a25-4ee0-bfe7-00a0fa6b9547', '73a0cd62-b7d0-473b-8568-8884135c4925']}


In [ ]:
from HUGS.Client import Process

processing = Process(service_url="%s/hugs" % base_url)
result = processing.process_file(auth=auth, par=par, par_secret=par_secret, data_type="CRDS")
print(result)

['7272bd9a-62e7-4c12-a9b2-30c2e8e136f0', '24373428-1a25-4ee0-bfe7-00a0fa6b9547', '73a0cd62-b7d0-473b-8568-8884135c4925', '8077f1ad-33d4-4365-a8d3-65b55edc96a5', 'f56a1149-7c12-45ce-9977-57f5ba840ee2', 'f7c8dcc7-2e25-4f58-8f76-138c7699dd63', 'dfe91850-f6be-497d-8e92-2de74474e3e8', '20719e06-3b2f-4e44-92f6-d905a649644c', 'e7053cdb-5e8c-4ed7-a1e1-4f4b829f274f', '197d764d-5045-4d7e-8927-ae7936a9a0a1', '31dbcf53-20b2-4026-906e-7c5a420190ec', 'aa1eda2b-faa0-4807-afaf-5aa778f9353f']


In [ ]:
def process_folder():
    """ 
    
    """

{'results': ['7272bd9a-62e7-4c12-a9b2-30c2e8e136f0',
  '24373428-1a25-4ee0-bfe7-00a0fa6b9547',
  '73a0cd62-b7d0-473b-8568-8884135c4925']}

In [ ]:
from HUGS.Processing import search

search_results = None

search_terms = widgets.Text(
                    value="",
                    placeholder = "Search",
                    description = "Search terms:",
                    disabled = False
)

locations = widgets.Text(
                value="",
                placeholder = "BSD, HFD",
                description = "Locations:",
                disabled = False
)

data_type = widgets.Dropdown(
                options=["CRDS", "GC"], 
                value="CRDS",
                description="Data type",
                disabled=False
    )



layout=widgets.Layout(display = "flex", width = "50%")
search_button = widgets.Button(description="Search", button_style="success", \
                               layout=widgets.Layout(flex='1 1 0%', width='25%'))

start_picker = widgets.DatePicker(
                    description='Start date',
                    disabled=False
)

end_picker = widgets.DatePicker(
                    description='End date',
                    disabled=False
)


box = widgets.VBox(children=[search_terms, locations, start_picker, end_picker, data_type,
                                 search_button], layout=layout)



def call_search(x):
    """ Call the search function and pass it the values 
        in the text boxes
            
    """
    from datetime import datetime
    from Acquire.ObjectStore import datetime_to_string
    start = datetime(1970,1,1) #datetime.combine(start_picker.value, datetime.min.time())
    end = datetime.now() #datetime.combine(end_picker.value, datetime.min.time())
    
    args = {}
    args["search_terms"] = search_terms.value
    
    # Remove whitespace and commas from the search terms
    split_search_terms = search_terms.value.replace(" ", "").split(",")
    args["search_terms"] = split_search_terms

    split_locations = locations.value.replace(" ", "").split(",")
    args["locations"] = split_locations
      
    args["locations"] = locations.value
    args["data_type"] = data_type.value
    args["start_datetime"] = datetime_to_string(start)
    args["end_datetime"] = datetime_to_string(end)
    
    global search_results
    search_results = hugs.call_function(function="search", args=args)
    
    print(search_results)
    
search_button.on_click(call_search)




In [ ]:
box

In [ ]:
def parse_results(results):
    dates = {}
    # Read the start and end date of the data found from key keys
    for search_key in results:
        sorted_keys = sorted(results[search_key])
        end_of_key = sorted_keys[0].split("/")[-1]
        
        start = end_of_key.split("_")[0]
        end = end_of_key.split("_")[-1]
        
        start = start.replace("T", " ")
        end = end.replace("T", " ")
        
        dates[search_key] = (start, end)
        
    return dates

In [ ]:
parse_results(search_results["results"])

In [ ]:
keys = search_results["results"]
data_args = {"return_type":"json", "keys": keys}
# Retrieve the data from the object store
data = hugs.call_function(function="retrieve", args=data_args)

In [ ]:
data

Now we can plot some of the data

In [ ]:
import pandas as pd
# co2_data = pd.read_json(data["results"]["bsd_co2"])
co_data = pd.read_json(data["results"]["bsd_ch4"])


In [ ]:
co_data.head(10)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

fig = plt.figure()
# ax = fig.add_subplot(211)
# ax.plot(co2_data.index.values, co2_data["co2 count"], color = "#4e79a7" )
ax = fig.add_subplot(111)
ax.plot(co_data.index.values, co_data["ch4 count"], color = "#59a14f")

plt.show()

In [ ]:
# Checkboxes for selecting which data we want to plot
# Select the gas, it goes and plots the 
# Simple line plot

In [ ]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster, TileLayer, ImageOverlay, GeoJSON,
    Polyline, Polygon, Rectangle, Circle, CircleMarker, Popup,
    SplitMapControl, WidgetControl,
    basemaps, basemap_to_tiles
)

from ipywidgets import HTML

In [ ]:
center = [54.2361, -4.548]
zoom = 5
m = Map(center=center, zoom=zoom)
mark_bsd = Marker(location=(54.942544, -1.369204))
mark_bsd.popup = HTML(value="Bilsdale")

mark_mhd = Marker(location=(53.20,-9.54))
mark_mhd.popup = HTML(value="Macehead")

mark_tac = Marker(location=(52.511, 1.155003))
mark_tac.popup = HTML(value="Tacolneston")

m += mark_bsd
m += mark_mhd
m += mark_tac

display(m)

In [ ]:
m